# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [10]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests
from pathlib import Path
from IPython.display import HTML
import folium
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
import numpy as np
from branca.element import Template, MacroElement
import json
import requests

# Import API key
from api_keys import geoapify_key

In [11]:
pwd

'c:\\Users\\wware\\Desktop\\UWA Bootcamp\\Challenges\\python-api-challenge\\VacationPy'

In [12]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
csv_file = Path("Resources/cities.csv")
city_data_df = pd.read_csv(csv_file)

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,fortaleza,-3.7227,-38.5247,28.07,78,0,2.57,BR,1715472798
1,1,neiafu,-18.6500,-173.9833,27.38,54,40,5.66,TO,1715472799
2,2,invercargill,-46.4000,168.3500,10.57,99,92,1.79,NZ,1715472800
3,3,longyearbyen,78.2186,15.6401,-0.09,80,75,4.63,SJ,1715472801
4,4,stanley,54.8680,-1.6985,11.40,89,0,2.06,GB,1715472801


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [4]:
# Get unique cities and assign a color to each - with hover data in this code
unique_cities = city_data_df['City'].unique()
colors = plt.cm.tab20(np.linspace(0, 1, len(unique_cities)))  # Using tab20 colormap
color_map = {city: mcolors.to_hex(color) for city, color in zip(unique_cities, colors)}

# Calculate mean latitude and longitude to center the map
mean_lat = city_data_df['Lat'].mean()
mean_lng = city_data_df['Lng'].mean()

# Create the Folium map with specified bounds
map = folium.Map(location=[mean_lat, mean_lng], zoom_start=2, tiles='CartoDB positron',
                 min_zoom=2, max_bounds=True)

# Add markers to the map without marker lines
for index, row in city_data_df.iterrows():
    popup_text = f"{row['City']}, {row['Country']}<br>Humidity: {row['Humidity']}"
    folium.CircleMarker(
        location=[row['Lat'], row['Lng']],
        radius=row['Humidity'] / 15,  # Adjust radius if needed
        color="none",  # This removes the stroke from the circle
        fill=True,
        fill_color=color_map[row['City']],
        fill_opacity=0.8,  # Set transparency level
        popup=folium.Popup(popup_text, max_width=200, sticky=True)  # Popup acts like hover
    ).add_to(map)

map

In [13]:
# Configure the map plot
map_plot = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo=True,
    tiles="OSM",
    frame_width=700,
    frame_height=500,
    size="Humidity",
    scale=1,
    color="City",
    #legend=False, # Hide the legend
    alpha=0.8
)

# Display the map
map_plot


:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [14]:
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,fortaleza,-3.7227,-38.5247,28.07,78,0,2.57,BR,1715472798
1,1,neiafu,-18.6500,-173.9833,27.38,54,40,5.66,TO,1715472799
2,2,invercargill,-46.4000,168.3500,10.57,99,92,1.79,NZ,1715472800
3,3,longyearbyen,78.2186,15.6401,-0.09,80,75,4.63,SJ,1715472801
4,4,stanley,54.8680,-1.6985,11.40,89,0,2.06,GB,1715472801


In [15]:
# Narrow down cities that fit criteria
# Filter the DataFrame based on the conditions
filtered_cities_df = city_data_df[
    (city_data_df['Max Temp'] > 21) & 
    (city_data_df['Max Temp'] < 27) & 
    (city_data_df['Wind Speed'] < 4.5) & 
    (city_data_df['Cloudiness'] == 0)
]

# Display the filtered DataFrame
print(filtered_cities_df)


     City_ID             City      Lat       Lng  Max Temp  Humidity  \
127      127           robore -18.3333  -59.7500     26.80        45   
188      188     poplar bluff  36.7570  -90.3929     25.88        38   
212      212        andalusia  31.3085  -86.4833     23.97        58   
226      226  remire-montjoly   4.9167  -52.2667     26.02        94   
254      254       monticello  45.3055  -93.7941     25.18        23   
307      307       pocahontas  36.2615  -90.9712     25.95        33   
309      309           keokuk  40.3973  -91.3849     23.29        33   
371      371           sparta  43.9441  -90.8129     21.09        30   
390      390     saint-pierre -21.3393   55.4781     21.82        60   
414      414           maxixe -23.8597   35.3472     21.97        88   
432      432          emporia  38.4039  -96.1817     26.11        34   
444      444         holualoa  19.6228 -155.9522     26.25        74   
463      463             tete -16.1564   33.5867     24.05      

In [18]:
# Drop any results with ull values
# Drop any rows with null values in the columns of interest
filtered_cities_df.dropna(subset=['Max Temp', 'Wind Speed', 'Cloudiness'], inplace=True)

# Display the cleaned DataFrame to confirm removal of null values
print(filtered_cities_df)


     City_ID             City      Lat       Lng  Max Temp  Humidity  \
127      127           robore -18.3333  -59.7500     26.80        45   
188      188     poplar bluff  36.7570  -90.3929     25.88        38   
212      212        andalusia  31.3085  -86.4833     23.97        58   
226      226  remire-montjoly   4.9167  -52.2667     26.02        94   
254      254       monticello  45.3055  -93.7941     25.18        23   
307      307       pocahontas  36.2615  -90.9712     25.95        33   
309      309           keokuk  40.3973  -91.3849     23.29        33   
371      371           sparta  43.9441  -90.8129     21.09        30   
390      390     saint-pierre -21.3393   55.4781     21.82        60   
414      414           maxixe -23.8597   35.3472     21.97        88   
432      432          emporia  38.4039  -96.1817     26.11        34   
444      444         holualoa  19.6228 -155.9522     26.25        74   
463      463             tete -16.1564   33.5867     24.05      

C:\Users\wware\AppData\Local\Temp\ipykernel_40528\1873084615.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_cities_df.dropna(subset=['Max Temp', 'Wind Speed', 'Cloudiness'], inplace=True)


In [19]:
# Display sample data
filtered_cities_df.head(10)

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
127,127,robore,-18.3333,-59.7500,26.80,45,0,2.01,BO,1715472905
188,188,poplar bluff,36.7570,-90.3929,25.88,38,0,2.06,US,1715472962
212,212,andalusia,31.3085,-86.4833,23.97,58,0,2.57,US,1715472935
226,226,remire-montjoly,4.9167,-52.2667,26.02,94,0,0.00,GF,1715472994
254,254,monticello,45.3055,-93.7941,25.18,23,0,1.54,US,1715473017
307,307,pocahontas,36.2615,-90.9712,25.95,33,0,3.60,US,1715473059
309,309,keokuk,40.3973,-91.3849,23.29,33,0,3.60,US,1715473060
371,371,sparta,43.9441,-90.8129,21.09,30,0,0.00,US,1715473114
390,390,saint-pierre,-21.3393,55.4781,21.82,60,0,4.12,RE,1715473132
414,414,maxixe,-23.8597,35.3472,21.97,88,0,0.66,MZ,1715473152


### Step 3: Create a new DataFrame called `hotel_df`.

In [22]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
# Create a new DataFrame with selected columns
hotel_df = filtered_cities_df[['City', 'Country', 'Lat', 'Lng', 'Humidity']].copy()

# Display the initial data
print(hotel_df.head())

                City Country      Lat      Lng  Humidity
127           robore      BO -18.3333 -59.7500        45
188     poplar bluff      US  36.7570 -90.3929        38
212        andalusia      US  31.3085 -86.4833        58
226  remire-montjoly      GF   4.9167 -52.2667        94
254       monticello      US  45.3055 -93.7941        23


In [23]:
# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
# Add an empty column for "Hotel Name"
hotel_df['Hotel Name'] = ""

# Display the DataFrame to confirm the new column
print(hotel_df.head())

                City Country      Lat      Lng  Humidity Hotel Name
127           robore      BO -18.3333 -59.7500        45           
188     poplar bluff      US  36.7570 -90.3929        38           
212        andalusia      US  31.3085 -86.4833        58           
226  remire-montjoly      GF   4.9167 -52.2667        94           
254       monticello      US  45.3055 -93.7941        23           


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [24]:
pwd

'c:\\Users\\wware\\Desktop\\UWA Bootcamp\\Challenges\\python-api-challenge\\VacationPy'

In [25]:
import requests
from api_keys import geoapify_key

url = 'https://api.geoapify.com/v2/places' 
params = {
    'categories': 'accommodation.hotel',
    'filter': 'circle:-122.4194,37.7749,10000',
    'limit': 1,
    'apiKey': geoapify_key
}

response = requests.get(url, params=params)
response.json()  # This will print the response from the API


{'type': 'FeatureCollection',
 'features': [{'type': 'Feature',
   'properties': {'name': 'Hotel Kabuki',
    'country': 'United States',
    'country_code': 'us',
    'state': 'California',
    'city': 'San Francisco',
    'postcode': '94115',
    'district': 'Western Addition',
    'street': 'Laguna Street',
    'housenumber': '1625',
    'lon': -122.42860829440426,
    'lat': 37.7854004,
    'state_code': 'CA',
    'formatted': 'Hotel Kabuki, 1625 Laguna Street, San Francisco, CA 94115, United States of America',
    'address_line1': 'Hotel Kabuki',
    'address_line2': '1625 Laguna Street, San Francisco, CA 94115, United States of America',
    'categories': ['accommodation',
     'accommodation.hotel',
     'building',
     'building.accommodation',
     'internet_access'],
    'details': ['details',
     'details.contact',
     'details.facilities',
     'details.wiki_and_media'],
    'datasource': {'sourcename': 'openstreetmap',
     'attribution': '© OpenStreetMap contributors'

In [26]:
# Set radius in meters for the hotel search around the city coordinates
radius = 10000  # 10,000 meters

print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # Get latitude and longitude from the DataFrame
    lat, lng = row['Lat'], row['Lng']

    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params = {
        'categories': 'accommodation.hotel',
        'filter': f'circle:{lng},{lat},{radius}',
        'limit': 1,
        'apiKey': geoapify_key
    }

    # Set base URL for the Geoapify Places API
    base_url = "https://api.geoapify.com/v2/places"

    # Make an API request using the params dictionary
    response = requests.get(base_url, params=params)

    # Convert the API response to JSON format
    hotels_response = response.json()

    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels_response["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{row['City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data to confirm updates
print(hotel_df.head())


Starting hotel search
robore - nearest hotel: La Casona de Don Pepe
poplar bluff - nearest hotel: Holiday Inn
andalusia - nearest hotel: Best Western
remire-montjoly - nearest hotel: Central Hôtel
monticello - nearest hotel: Best Western Plus Chelsea Hotel
pocahontas - nearest hotel: No hotel found
keokuk - nearest hotel: Quality Inn & Suites
sparta - nearest hotel: Country Inn & Suites
saint-pierre - nearest hotel: La Villa Delisle
maxixe - nearest hotel: Casa Jensen
emporia - nearest hotel: Budget Host Inn Emporia
holualoa - nearest hotel: Kona Seaside Hotel
tete - nearest hotel: Tete Ferry Sun
dalbandin - nearest hotel: SF Al-dawood Restaurant
inhambane - nearest hotel: Casa Jensen
pacific grove - nearest hotel: InterContinental The Clement Monterey Hotel
                City Country      Lat      Lng  Humidity  \
127           robore      BO -18.3333 -59.7500        45   
188     poplar bluff      US  36.7570 -90.3929        38   
212        andalusia      US  31.3085 -86.4833     

In [27]:
hotels=hotel_df['Hotel Name'].nunique()
hotels

15

### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [28]:
import hvplot.pandas  # Ensure hvplot is imported if not already

# Filter out rows where 'No hotel found'
filtered_hotel_df = hotel_df[hotel_df['Hotel Name'] != 'No hotel found']

# Configure the map plot using filtered DataFrame with additional hover information
map_plot = filtered_hotel_df.hvplot.points(
    "Lng",
    "Lat",
    geo=True,
    tiles="OSM",
    frame_width=700,
    frame_height=500,
    size='Humidity',
    scale=1,
    color='City',
    alpha=0.8,
    hover_cols=['Hotel Name', 'Country']  # Additional columns to display in hover tooltip
)

# Display the map
map_plot


:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name,Country)